# Colab Custom Magnet

This is an example of uploading one or more STL files to Colab, and then running the magnetic field calculation in the cloud. 

In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab. Installing pymagnet package from pypi')
    %pip install pymagnet -q
    from google.colab import files
    print("===============================================")
    print("Please browse and upload one or more stl files:")
    uploaded = files.upload() 
else:
    print('Not running on CoLab.')

## Import the needed modules

If there is a `NumbaWarning` about the TBB threading layer, this can be safely ignored. The default version of TBB on COLAB doesn't support parallelised Numba functions. This applies only to STL `MeshMagnet` types, where the field calculation routine is run in serial mode instead 

In [ ]:
%matplotlib inline
# %matplotlib notebook
%config InlineBackend.figure_format = 'retina'
import pymagnet as pm
import numpy as np
import pandas as pd

## Prepare magnet helper function

In [ ]:
def gen_mesh_mag(file, Jr=1.0, theta=0, phi=0, alpha=0, beta=0, gamma=0):
    pm.reset()
    center = (0, 0, 0)
    m_mesh = pm.magnets.Mesh(
        file,
        Jr=Jr,
        center=center,
        theta=theta,
        phi=phi,
        alpha=alpha,
        beta=beta,
        gamma=gamma,
    )
    return m_mesh


The below code will import all uploaded files, and list them:

In [ ]:
for file in uploaded.keys():
  if file[-3:].lower() == 'stl':
    print(file)

Choose one file and render it:

**WARNING**

If the file has more than 500 simplexes the calcuation will take more than a few seconds. This is because the outer loops over magnet and simplex occur in Python, while the actual field calculations for each simplex at all points is done at a lower level thanks to Numba and Numpy.

This limitation will be redressed with the move to Rust for the backend.


In [ ]:
example_file = list(uploaded.keys())[0]


mesh_magnet = gen_mesh_mag(
    file = example_file,
    Jr=1.0,
    theta=0,
    phi=0,
    alpha=0,
    beta=-90,
    gamma=0,
)

# Useful for setting automatic ranges on field calculations below
max_x = mesh_magnet.mesh_vectors[:,:,0].max()
max_y = mesh_magnet.mesh_vectors[:,:,1].max()
max_z = mesh_magnet.mesh_vectors[:,:,2].max()
max_all = np.max([max_x, max_y, max_z])


print(f"{example_file} has {len(mesh_magnet.mesh_vectors)} simplexes")

fig = pm.plots.plot_magnet()

Calculate and plot the field along three principal planes using the `slice_quickplot` convenience function:

In [ ]:
fig, slice_cache, data_objects = pm.plots.slice_quickplot(
    cmax=0.5,
    num_levels=6,
    num_points=100,
    opacity=0.7,
    num_arrows=10,
    cone_opacity=0.3,
    show_magnets=True,
    max1=np.around(max_all*1.3),
    max2=np.around(max_all*1.3),
    slice_value=0.0,
)

The calcualtions are stored in the `slice_cache`, meaning we can replot this data as 2D contour slices can be generated for each plane in the generated data

In [ ]:
for plane in slice_cache.keys():
    pm.plots.plot_3D_contour(slice_cache[plane]['points'], slice_cache[plane]['field'], plane,
                             cmin = 0,
                             cmax=0.5,
                             num_levels=6,
#                              num_arrows = 20,
#                              vector_color = 'k'
                            )

Or replot the 3D data:

In [ ]:
fig=pm.plots.slice_plot(slice_cache,
                        cmin = 0,
                        cmax=0.3,
                        num_levels=7,
                        cmap='viridis',
                        opacity = 0.8,
#                              num_arrows = 20,
#                              vector_color = 'k'
                        )

Saving can be done using your preferred method, a simple but inefficient way is to use the hdf capabilities of pandas:

In [ ]:
save_file = example_file[:-4] + "slices.h5"

print(f"Saving to: {save_file}")
cache_df = pd.DataFrame(slice_cache)
cache_df.to_hdf(save_file, key=example_file[:-4]) 

Volume plots can be quickly calculated and rendered using the `volume_quickplot` method:

In [ ]:
fig_vol, vol_cache, data_objects = pm.plots.volume_quickplot(cmin=0.0,
                                    cmax=0.4,
                                    opacity=0.3,
                                    magnet_opacity=1.0,
                                    num_levels=6,
                                    num_points=30,
                                    show_magnets = True,
                                    xmax = np.around(max_all*1.3),
                                    ymax = np.around(max_all*1.3),
                                    zmax = np.around(max_all*1.3),
                                    unit = 'mm',
                                    opacityscale = 'max',
                                   )

With the resultant data, `vol_cache` being replotted using `volume_plot`


In [ ]:
fig=pm.plots.volume_plot(vol_cache['points'], vol_cache['field'],
                         cmin = 0,
                         cmax=0.3,
                         num_levels=7,
                         opacity = 0.3,
                         magnet_opacity=0.8,
                         vector_arrows = 20,
                         opacityscale='max'
                        )